# TODO : create a parcellation from the average of the similatriy matrix,

In [ ]:
from pathlib import Path
import numpy as np
import nibabel as nib 
import networkx as nx

from similarity_matrix import compute_similarity_matrix_pca
from smoothing import smooth_surface_graph
from gradient import compute_gradients, build_mesh_graph
from watershed import watershed_by_flooding
from visualization import visualize_brain_surface

In [ ]:
# # Path Run all subject lh on run 1

# config = {
# "fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
# "subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N20")
# }

# hemisphere = 'lh'
# run = 2

# surface_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.white"
# surface_inf_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.inflated"
# # Extract the Surface Mesh
# coords, faces = nib.freesurfer.read_geometry(str(surface_path))
# coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
# graph = build_mesh_graph(faces)

# # Output paths
# output_gradient_dir = config["subjects_dir"] / "gradient"
# (output_gradient_dir).mkdir(exist_ok=True, parents=True)
# output_labels_dir = config["subjects_dir"] / "labels"
# (output_labels_dir).mkdir(exist_ok=True, parents=True)



# # Variables path
# subject = f"{1:02d}"
# subj_dir = config["subjects_dir"] / f"sub-{subject}"

# # Define paths using pathlib
# surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
# vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
# brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"


# # If needed, the grey matter mask
# # gm_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / f"{hemisphere}.ribbon.mgz"


# print('all paths defined')

In [ ]:
from nilearn.image import resample_img
def load_data_normalized(surf_fmri_path,
                         vol_fmri_path, 
                         brain_mask_path):

    # Load the surface data
    surf_fmri = nib.load(str(surf_fmri_path)).get_fdata().squeeze()
    surf_fmri_n = (surf_fmri - np.mean(surf_fmri, axis=1, keepdims=True)) / np.std(surf_fmri, axis=1, keepdims=True)
    
    # Load the images using nibabel
    vol_fmri = nib.load(str(vol_fmri_path))
    mask_img = nib.load(brain_mask_path)

    # resample the mask to the right one
    mask_img = resample_img(
        mask_img,
        target_affine=vol_fmri.affine,
        target_shape=vol_fmri.get_fdata().shape[:-1],
        interpolation='nearest',
        force_resample=True
    )

    fmri_data = vol_fmri.get_fdata()
    mask_data = mask_img.get_fdata().astype(bool)  # Convert mask to boolean
    # Normalize the data
    vol_fmri = fmri_data[mask_data] 
    vol_fmri_n = (vol_fmri - np.mean(vol_fmri, axis=1, keepdims=True)) / np.std(vol_fmri, axis=1, keepdims=True)
    
    # Remove nans from the data
    surf_fmri_n = np.nan_to_num(surf_fmri_n).astype(np.float32)
    vol_fmri_n = np.nan_to_num(vol_fmri_n).astype(np.float32)
    return surf_fmri_n, vol_fmri_n

In [ ]:
# # Full pipeline my way
    
# # 1 : Compute the average similartiy matrix across each subjects
# sim_matrix_sum = np.zeros(nx.adjacency_matrix(graph).shape, dtype=np.float32)
# print('sim_matrix_sum shape', sim_matrix_sum.shape)
# for i in range(1,21):
#     if i == 5:
#         continue # Subject 5 is missing
#     # Variables path
#     subject = f"{i:02d}"
#     subj_dir = config["subjects_dir"] / f"sub-{subject}"

#     # Define paths using pathlib
#     surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
#     vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
#     brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"

#     # Load the data
#     surf_fmri_n, vol_fmri_n = load_data_normalized(surf_fmri_path,
#                                                     vol_fmri_path, 
#                                                     brain_mask_path)
    
#     sim_matrix_sum += compute_similarty_matrix_PCA(vol_fmri_n,
#                                                 surf_fmri_n)

# # Save the similarty matrix
# sim_matrix_path = config["subjects_dir"] / f"similarity_matrix_group_run{run}_{hemisphere}.npy"
# np.save(sim_matrix_path, sim_matrix_sum)
    
    
# # 2 : Smooth the similatry matrix
# sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix_sum, iterations=5)

# # 3 : Compute the gradient of the similarty matrix
# gradients = compute_gradients(graph, sim_matrix_smoothed)
# gradients_sum = gradients.sum(axis=1).astype(np.float32) # TODO : HERE is the diff between my way and gordon way

# # 3.1 : Smooth the gradient
# gradient_smoothed = smooth_surface_graph(graph, gradients_sum, iterations=10)
# gradients_path = config["subjects_dir"] / f"gradients_group_run{run}_{hemisphere}.npy"
# np.save(gradients_path, gradients)

# # 4 : Compute the watershed of each gradients to optain gradient maps
# labels = watershed_by_flooding(graph, gradient_smoothed)
# labels_path = config["subjects_dir"] / f"labels_group_run{run}_{hemisphere}.npy"
# np.save(labels_path, labels)

# # 5 : Visualize the watershed
# # visualize_brain_surface(coords_, faces_, gradient_smoothed)



In [ ]:
# Full pipeline GORDON WAY
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N18")
}
run = 2

for hemisphere in ['lh', 'rh']:
    
    surface_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.white"
    surface_inf_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.inflated"
    # Extract the Surface Mesh
    coords, faces = nib.freesurfer.read_geometry(str(surface_path))
    coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
    graph = build_mesh_graph(faces)

    # Output paths
    output_dir = config["subjects_dir"] / "group_parcellation"
    (output_dir).mkdir(exist_ok=True, parents=True)
    
    # 1 : Compute the average similartiy matrix across each subjects
    sim_matrix_sum = np.zeros(nx.adjacency_matrix(graph).shape, dtype=np.float32)
    print('sim_matrix_sum shape', sim_matrix_sum.shape)
    for i in range(1,19):
        # if i == 5:
        #     continue # Subject 5 is missing PPSFACE20
        
        # >>>>>>>>>>>>>>>>>>> To modify if needed
        # Variables path
        subject = f"{i:02d}"
        subj_dir = config["subjects_dir"] / f"sub-{subject}"

        # Define paths using pathlib
        surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
        vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
        brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"
        # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        
        # Load the data
        surf_fmri_n, vol_fmri_n = load_data_normalized(surf_fmri_path,
                                                       vol_fmri_path, 
                                                       brain_mask_path)
        
        sim_matrix_sum += compute_similarity_matrix_pca(vol_fmri_n,
                                                       surf_fmri_n)

    # Save the similarty matrix
    sim_matrix_path = output_dir / f"similarity_matrix_group_run{run}_{hemisphere}.npy"
    np.save(sim_matrix_path, sim_matrix_sum)
    
    
    # 2 : Smooth the similatry matrix
    sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix_sum, iterations=5)

    # 3 : Compute the gradient of the similarty matrix
    gradients = compute_gradients(graph, sim_matrix_smoothed)

    # 3.1 : Smooth the gradient
    gradients_smoothed = smooth_surface_graph(graph, gradients, iterations=10)
    gradients_path = output_dir / f"gradients_smoothed_group_run{run}_{hemisphere}.npy"
    np.save(gradients_path, gradients_smoothed)


    # 4 : Create the boundary map from the watershed
    boundary_map = np.zeros_like(gradients[:,0]).astype(np.float32)
    for map_idx in range(gradients.shape[1]): # loop over each columns of the matrix
        if map_idx % 10 == 0:
            print(f"Processing map {map_idx}") # Print the current map index
        boundary = (watershed_by_flooding(graph, gradients_smoothed[:,map_idx])<0)*1 # Extract the bounary from the watershed algorithm
        boundary_map += boundary
    
    boundary_map_path = output_dir / f"boundary_map_group_run{run}_{hemisphere}.npy"
    np.save(boundary_map_path, boundary_map)

    # Create the FINAL parcels
    bound_smoothed = smooth_surface_graph(graph, boundary_map, iterations=10)
    label_bound = watershed_by_flooding(graph, bound_smoothed)
    # Save the parcels
    np.save(output_dir / f"labels_group_run12_smooth10_run{run}_{hemisphere}.npy", label_bound)



In [ ]:
# WARNING BEFOR USING THIS SCRIPT
"""
This part is used in order to create a parcellation from the run1 and run2.
"""
ppsface = "PPSFACE_N20"
for hemi in ["lh", "rh"]:
   surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
   inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
   # surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
   coords, faces = nib.freesurfer.read_geometry(surface_path)
   coords_, faces_ = nib.freesurfer.read_geometry(inf_path)
   graph = build_mesh_graph(faces)
   bound_sum = 0
   for run in ["1","2"]:
      # Extract the parcellation
      bound_sum += np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\\boundary_map_group_run{run}_{hemi}.npy")
        
   # Create the parcels
   bound_smoothed = smooth_surface_graph(graph, bound_sum, iterations=10)
   label_bound = watershed_by_flooding(graph, bound_smoothed)
   # Save the parcels
   np.save(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_{hemi}.npy", label_bound)

## Show the Results

In [ ]:
# 5 : Visualize the bounardy map
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N20")
}
run = 2
surface_inf_path = config["fsavg6_dir"] / "surf" / f"lh.inflated"
surface_path = config["fsavg6_dir"] / "surf" / f"lh.white"

coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
coords, faces = nib.freesurfer.read_geometry(str(surface_path))
graph = build_mesh_graph(faces)

path_bound1 = r"D:\Data_Conn_Preproc\PPSFACE_N20\group_parcellation\boundary_map_group_run1_lh.npy"
path_bound2 = r"D:\Data_Conn_Preproc\PPSFACE_N20\group_parcellation\boundary_map_group_run2_lh.npy"
boundary_map1 = np.load(path_bound1)
boundary_map2 = np.load(path_bound2)

# put the 2 runs together
boundary_map = boundary_map1 

print('boundary_map shape', boundary_map1.shape)


In [ ]:
visualize_brain_surface(coords_, faces_, boundary_map, title='Bounardy Map lh PPSFACE20')

In [ ]:
bound_smoothed = smooth_surface_graph(graph, boundary_map, iterations=10)
visualize_brain_surface(coords_, faces_, bound_smoothed)

In [ ]:
labels = watershed_by_flooding(graph, bound_smoothed)
visualize_brain_surface(coords_, faces_, labels, title="Group-Level Parcellation lh PPSFACE20")